In [1]:
#imports
import logging
import torch
# set seed
import pandas as pd
import numpy as np
import utils as ut
import experiment as ex
from evaluation import *
from sklearn.metrics import mean_squared_error
from torch.utils.tensorboard import SummaryWriter
import torch
from river.neighbors import KNNRegressor
import random
#define fixed_hyperparams and create a config gen
from configurations import RandomConfigGen, Configuration
from torch import nn
from deep_net import RandomNet
from experiment import run_experiment
import regex as re
from pathlib import *
#from sk_models import PLSRegressor
import matplotlib.pyplot as plt

from sk_models import PLSRegression, StandardScaler,LocalWeightedRegression,PLSLWR,LinearRidge
from river_models import *

from river import stream,linear_model,preprocessing, ensemble, metrics, optim
from river.neighbors import KNNRegressor 
from sklearn.metrics import mean_squared_error
from river.utils import dict2numpy, numpy2dict

seed = 1
torch.manual_seed(seed)
random.seed(seed + 1)
np.random.seed(seed + 2)
random_state = np.random.RandomState(seed)
import gc
torch.cuda.empty_cache()
gc.collect()

print(f"GPU detected is {torch.cuda.get_device_name(0)}")

GPU detected is GeForce GTX 970


In [2]:
#setup input and output directories

#setup input and outpu t formats, load data

#we need to set parametesr
file_name = "PLN7.csv" #"mango_684_990.csv" #"mango_729_975.csv" #fitlered=513-1050
id_cols =["db_id","sample_id"] #
output_cols = None
data_path = Path('D:/workspace/lazydeep/data/soil_data/')
log_path = Path("D:/workspace/lazydeep/experiments/10.3") #1.01/")
if not log_path.exists():
    log_path.mkdir()

data_file = data_path / file_name
log_dir = log_path / re.sub(r'\.(?=csv$)[^.]+$', '',file_name)
if not log_dir.exists():
    log_dir.mkdir()
print(f"Output directory is {log_dir}")

Output directory is D:\workspace\lazydeep\experiments\10.3\PLN7


In [3]:
data = pd.read_csv(data_file)
data=data.sample(frac=1,random_state=random_state)

pre_ind =[i for i in range(0,10000)]
pretrain_ind,pretest_ind = train_test_split(pre_ind,train_size=5/6,random_state=random_state,shuffle=False)
stream_ind = [i for i in range(10000,110000)]

pretrain_data =  ut.TabularDataset(data.iloc[pretrain_ind,:],id_cols = id_cols, cat_cols=None, output_cols=output_cols, ignore_cols= None)
pretest_data = ut.TabularDataset(data.iloc[pretest_ind,:],id_cols = id_cols, cat_cols=None, output_cols=output_cols, ignore_cols= None)
stream_data = ut.TabularDataset(data.iloc[stream_ind,:],id_cols = id_cols, cat_cols=None, output_cols=output_cols, ignore_cols= None)

nrow, ncol = data.shape
nrow_train = len(pretrain_data)
nrow_test = len(pretest_data)
nrow_stream = len(stream_data)

print(f"train: {nrow_train}, test: {nrow_test}, stream: {nrow_stream}")

train: 8333, test: 1667, stream: 100000


In [4]:
#deep, deep_lwr, ensemble_deep, deep_ensemble_lwr, ensemble_deep

pls_nums = ["random_82","random_24","random_10","random_4","random_73"]
deep_nums = ["random_29","random_60","random_63","random_41","random_15"]  #[13,57,77,60,76]

pls_model_dir = Path("D:/workspace/lazydeep/experiments/1.01/PLN7")
deep_model_dir = Path("D:/workspace/lazydeep/experiments/2.00/PLN7")

pls_scaler = PLSRegression(n_components=34).from_state(PLSRegression(n_components=34).load_state(pls_model_dir/'preprocessing'/f"_final"))      
deep_scaler = StandardScaler().from_state(StandardScaler().load_state(deep_model_dir/'preprocessing'/f"_final"))      

def build_model(dir_,id_,scaler_):
    deep_ = torch.load(dir_/"models"/id_/"_model")
    deep_.load_state(dir_/"models"/id_/"_final")
    
    return (StreamWrapper(scaler_)|StreamDeep(deep_))

def build_model_lwr(dir_,id_,scaler_,k_=1000):
    model = build_model(dir_,id_,scaler_)
    
    return (model|StreamLocalWeightedRegression(n_neighbors=k_,floor=True))

def build_model_knn(dir_,id_,scaler_,k_=5):
    model = build_model(dir_,id_,scaler_)
    
    return (model|KNNRegressor(n_neighbors=k_))



In [5]:
river_models = {}

#build deep models
for i,k in enumerate(pls_nums):
    river_models[f'pls-deep{i}'] = build_model(pls_model_dir,k,pls_scaler)
for i,k in enumerate(deep_nums):
    river_models[f'std-deep{i}'] = build_model(deep_model_dir,k,deep_scaler)
    
#build deep lwr models
for i,k in enumerate(pls_nums):
    river_models[f'pls-deep{i}-knn'] = build_model_knn(pls_model_dir,k,pls_scaler)
for i,k in enumerate(deep_nums):
    river_models[f'std-deep{i}-knn'] = build_model_knn(deep_model_dir,k,deep_scaler)



#setup our metrics and stores of results
full_set = river_models.keys()
metrics = {'R2':{name:metrics.R2() for name in full_set},
           'R2_rolling':{name:RollingR2(window_size=1000) for name in full_set},
           'MSE':{name:metrics.MSE() for name in river_models.keys()},
           'MSE_rolling':{name:RollingMSE(window_size=1000) for name in full_set}
          }

In [6]:
full_set = river_models.keys()
metrics = {'R2':{name:metrics.R2() for name in full_set},
           'R2_rolling': {name:RollingR2(window_size=1000) for name in full_set},
           'MSE':{name:metrics.MSE() for name in river_models.keys()},
           'MSE_rolling':{name:RollingMSE(window_size=1000) for name in full_set}
          }

In [7]:
#take our pretrained models, now evaluate them on 

In [8]:
_,_,river_models,metrics = prequential_evaluate(pretrain_data,river_models,metrics,pretrain = len(pretrain_data),num_its=len(pretrain_data))

  0%|          | 0/8333 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
preds_test,scores_test,metrics = score_evaluate(pretest_data,river_models,metrics,num_its=len(pretest_data))

In [ ]:
preds_stream, scores_stream,river_models,metrics = prequential_evaluate(stream_data,river_models,metrics,pretrain=0,num_its=100000)

In [ ]:
def zip_dict(dict1,dict2):
 
    dict12 = {k:dict1[k]+dict2[k] for k in dict1.keys()}   
    return dict12

def zip_nested_dict(dict1,dict2):
    dict12 = {}
    
    for k in dict1.keys():
        dict12[k] = {name:dict1[k][name]+dict2[k][name] for name in dict1[k].keys()}
    return dict12

In [ ]:
preds = zip_dict(preds_test,preds_stream)

In [ ]:
scores = zip_nested_dict(scores_test,scores_stream)

In [ ]:
preds_df = pd.DataFrame(preds)

for (columnName, columnData) in preds_df.iteritems():
    preds_df[f'diff_{columnName}'] = columnData-preds_df['y']

preds_df.to_csv(log_dir/"preds_df.csv")

In [ ]:
#findings
#1) preprocessing works, random lr things for lr don't
#) standardisation asks as regularisation

scores_df_rolling = pd.DataFrame(scores['R2_rolling'])
scores_df_total = pd.DataFrame(scores['R2'])

In [ ]:
if False:
    fig, ax = plt.subplots()

    ax.set_xlabel("Stream Index")
    ax.set_ylabel("R^2 Score")
    ax.set_title("Streaming performance ")

    for (columnName, columnData) in scores_df.iteritems():

        ax.plot(columnData.index,columnData,'-',label = f"{columnName}")
    ax.plot([len(pretest_data),len(pretest_data)],[-1,1],c="black",ls='--')

    ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
    plt.savefig(log_dir / f"r2_plot.png",bbox_inches='tight')
    ax.set_ylim(0,1)
    plt.savefig(log_dir / f"r2_plot_v2.png",bbox_inches='tight')


In [ ]:
fig, ax = plt.subplots()

ax.set_xlabel("Stream Index")
ax.set_ylabel("R^2 Score")
ax.set_title("Streaming performance (rolling average) ")

scores_df = pd.DataFrame(scores['R2_rolling'])
scores_df.to_csv(log_dir/"R2_rolling.csv")
for (columnName, columnData) in scores_df.iteritems():
    ax.plot(columnData.index,columnData,label = f"{columnName}")
ax.plot([len(pretest_data),len(pretest_data)],[0,1000],c="black",ls='--')
ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
plt.savefig(log_dir / f"mse_plot_rolling.png",bbox_inches='tight')
plt.savefig(log_dir / f"mse_plot_rolling_v2.png",bbox_inches='tight')


fig, ax = plt.subplots()

ax.set_xlabel("Stream Index")
ax.set_ylabel("R^2 Score")
ax.set_title("Streaming performance (rolling average) ")
    
scores_df = pd.DataFrame(scores['R2_rolling'])
scores_df.to_csv(log_dir/"r2_scores_rolling.csv")
for (columnName, columnData) in scores_df.iteritems():
    columnData
    ax.plot(columnData.index,columnData,label = f"{columnName}")
ax.plot([len(pretest_data),len(pretest_data)],[-1,1],c="black",ls='--')

ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
plt.savefig(log_dir / f"r2_plot_rolling.png",bbox_inches='tight')
ax.set_ylim(0,1)
plt.savefig(log_dir / f"r2_plot_rolling_v2.png",bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots()

ax.set_xlabel("Stream Index")
ax.set_ylabel("MSE")
ax.set_title("Streaming performance ")

scores_df = pd.DataFrame(scores['MSE'])
scores_df.to_csv(log_dir/"MSE.csv")
for (columnName, columnData) in scores_df.iteritems():
    ax.plot(columnData.index,columnData,'-',label = f"{columnName}")
ax.plot([len(pretest_data),len(pretest_data)],[0,1000],c="black",ls='--')
ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
plt.savefig(log_dir / f"mse_plot.png",bbox_inches='tight')
ax.set_ylim(0,1000)
plt.savefig(log_dir / f"mse_plot_v2.png",bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots()

ax.set_xlabel("Stream Index")
ax.set_ylabel("MSE")
ax.set_title("Streaming performance (rolling average) ")

scores_df = pd.DataFrame(scores['MSE_rolling'])
scores_df.to_csv(log_dir/"MSE_rolling.csv")
for (columnName, columnData) in scores_df.iteritems():
    ax.plot(columnData.index,columnData,label = f"{columnName}")
ax.plot([len(pretest_data),len(pretest_data)],[0,1000],c="black",ls='--')
ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
plt.savefig(log_dir / f"mse_plot_rolling.png",bbox_inches='tight')
ax.set_ylim(-1,200)
plt.savefig(log_dir / f"mse_plot_rolling_v2.png",bbox_inches='tight')

In [ ]:
import scipy.stats
from sklearn.metrics import mean_squared_error, mean_absolute_error

fig, ax = plt.subplots()
ax.set_ylabel('y_pred')
ax.set_xlabel('y_true')

for (columnName, columnData) in preds_df.iteritems():
        if not columnName == 'y':
            ax.scatter(preds_df['y'],columnData,label = f"{columnName}",s=1)
            
            corr_coef = scipy.stats.pearsonr(columnData, preds_df['y'])
            #slope, intercept, r, p, stderr = scipy.stats.linregress(columnData, preds_df['y'])
            loss = mean_squared_error(preds_df['y'], columnData)
            mae = mean_absolute_error(preds_df['y'], columnData)    
            print(f"{columnName}, R^2 = {corr_coef[0]}, MSE = {loss}")
            
ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
ax.set_ylim(0,500)

In [ ]:
def take_subset_by_str(dataset,s,reverse=False):
    col_names = dataset.columns.tolist()
    if reverse:
        encoding = [i for i in col_names if not (s in i)]
    else:
        encoding = [i for i in col_names if (s in i)]
    return dataset[encoding]

def take_subset_by_re(dataset,s,reverse=False):
    col_names = dataset.columns.tolist()
    if reverse:
        encoding = [i for i in col_names if not s.match(i)]
    else:
        encoding = [i for i in col_names if s.match(i)]
    return dataset[encoding]

In [ ]:
if False:
    subset2 = take_subset_by_str(scores_df ,'pls')
    fig, ax = plt.subplots()

    ax.set_xlabel("Stream Index")
    ax.set_ylabel("R2")
    ax.set_title("Streaming performance (rolling average) ")


    for (columnName, columnData) in subset2.iteritems():
        ax.plot(columnData.index,columnData,label = f"{columnName.replace('pls_deep_',' ').replace('pls_deep','base_model')}",linewidth=1.5)
    ax.plot([len(pretest_data),len(pretest_data)],[-1,1],c="black",ls='--')

    ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
    plt.savefig(log_dir / f"r2_plot_pls.png",bbox_inches='tight')
    ax.set_ylim(0,1)
    plt.savefig(log_dir / f"r2_plot_v2_pls.png",bbox_inches='tight')
    

In [ ]:
if False:
    subset2 = take_subset_by_str(scores_df ,'pls',reverse=True)
    fig, ax = plt.subplots()

    ax.set_xlabel("Stream Index")
    ax.set_ylabel("R2")
    ax.set_title("Streaming performance (rolling average) ")


    for (columnName, columnData) in subset2.iteritems():
        ax.plot(columnData.index,columnData,label =  f"{columnName.replace('deep_',' ').replace('deep','base_model')}")
    ax.plot([len(pretest_data),len(pretest_data)],[-1,1],c="black",ls='--')

    ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
    plt.savefig(log_dir / f"r2_plot_deep.png",bbox_inches='tight')
    ax.set_ylim(0,1)
    plt.savefig(log_dir / f"r2_plot_v2_deep.png",bbox_inches='tight')


In [ ]:
import re 
if False:
    pp_opts = ['deep_lwr']
    ws_opts = ['_s1','_s2','_s3','_s4']
    for pp_opt in pp_opts:
        subset1 = take_subset_by_str(scores_df,pp_opt)

        for ws_opt in ws_opts:
            subset2 = take_subset_by_str(subset1,ws_opt)

            fig, ax = plt.subplots()

            ax.set_xlabel("Stream Index")
            ax.set_ylabel("R2")
            ax.set_title("Streaming performance (rolling average) ")


            for (columnName, columnData) in subset2.iteritems():
                ax.plot(columnData.index,columnData,label = f"{columnName}")
            ax.plot([len(pretest_data),len(pretest_data)],[-1,1],c="black",ls='--')

            ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
            #plt.savefig(log_dir / f"r2_plot_{pp_opt}{ws_opt}.png",bbox_inches='tight')
            ax.set_ylim(-1,1)
            #plt.savefig(log_dir / f"r2_plot_v2_{pp_opt}{ws_opt}.png",bbox_inches='tight')


In [ ]:
# deep subset
p = re.compile('(pls-deep\d)(?!-lwr)')
subset = take_subset_by_re(scores_df_total ,p,reverse=False)
fig, ax = plt.subplots()

ax.set_xlabel("Stream Index")
ax.set_ylabel("R2")
ax.set_title("Streaming performance ")


for (columnName, columnData) in subset.iteritems():
    ax.plot(columnData.index,columnData,label =  f"{columnName}")
ax.plot([len(pretest_data),len(pretest_data)],[-1,1],c="black",ls='--')

ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
plt.savefig(log_dir / f"r2_plot_pls-deep.png",bbox_inches='tight')
ax.set_ylim(0,1)
plt.savefig(log_dir / f"r2_plot_v2_pls-deep.png",bbox_inches='tight')

subset = take_subset_by_re(scores_df_rolling ,p,reverse=False)
fig, ax = plt.subplots()

ax.set_xlabel("Stream Index")
ax.set_ylabel("R2")
ax.set_title("Streaming performance (rolling average) ")


for (columnName, columnData) in subset.iteritems():
    ax.plot(columnData.index,columnData,label =  f"{columnName}")
#ax.plot([len(pretest_data),len(pretest_data)],[-1,1],c="black",ls='--')

ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
plt.savefig(log_dir / f"r2_plot_rolling_pls-deep.png",bbox_inches='tight')
ax.set_ylim(0,1)
plt.savefig(log_dir / f"r2_plot_rolling_v2_pls-deep.png",bbox_inches='tight')

In [ ]:
p = re.compile('(std-deep\d)(?!-lwr)')
subset = take_subset_by_re(scores_df_total ,p,reverse=False)

fig, ax = plt.subplots()
ax.set_xlabel("Stream Index")
ax.set_ylabel("R2")
ax.set_title("Streaming performance")


for (columnName, columnData) in subset.iteritems():
    ax.plot(columnData.index,columnData,label =  f"{columnName}")
#ax.plot([len(pretest_data),len(pretest_data)],[-1,1],c="black",ls='--')

ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
plt.savefig(log_dir / f"r2_plot_std-deep.png",bbox_inches='tight')
ax.set_ylim(0,1)
plt.savefig(log_dir / f"r2_plot_v2_std-deep.png",bbox_inches='tight')

subset = take_subset_by_re(scores_df_rolling ,p,reverse=False)

fig, ax = plt.subplots()
ax.set_xlabel("Stream Index")
ax.set_ylabel("R2")
ax.set_title("Streaming performance (rolling average) ")


for (columnName, columnData) in subset.iteritems():
    ax.plot(columnData.index,columnData,label =  f"{columnName}")
#ax.plot([len(pretest_data),len(pretest_data)],[-1,1],c="black",ls='--')

ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
plt.savefig(log_dir / f"r2_plot_rolling_std-deep.png",bbox_inches='tight')
ax.set_ylim(0,1)
plt.savefig(log_dir / f"r2_plot_rolling_v2_std-deep.png",bbox_inches='tight')

In [ ]:
# pls-deep-lwr subset

p = re.compile('(pls-deep\d-knn)')
subset = take_subset_by_re(scores_df_total ,p,reverse=False)
fig, ax = plt.subplots()

ax.set_xlabel("Stream Index")
ax.set_ylabel("R2")
ax.set_title("Streaming performance ")


for (columnName, columnData) in subset.iteritems():
    ax.plot(columnData.index,columnData,label =  f"{columnName}")
#ax.plot([len(pretest_data),len(pretest_data)],[-1,1],c="black",ls='--')

ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
plt.savefig(log_dir / f"r2_plot_pls-deep-lwr.png",bbox_inches='tight')
ax.set_ylim(0,1)
plt.savefig(log_dir / f"r2_plot_v2_pls-deep-lwr.png",bbox_inches='tight')

subset = take_subset_by_re(scores_df_rolling ,p,reverse=False)
fig, ax = plt.subplots()

ax.set_xlabel("Stream Index")
ax.set_ylabel("R2")
ax.set_title("Streaming performance (rolling average) ")


for (columnName, columnData) in subset.iteritems():
    ax.plot(columnData.index,columnData,label =  f"{columnName}")
#ax.plot([len(pretest_data),len(pretest_data)],[-1,1],c="black",ls='--')

ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
plt.savefig(log_dir / f"r2_plot_rolling_pls-deep-lwr.png",bbox_inches='tight')
ax.set_ylim(0,1)
plt.savefig(log_dir / f"r2_plot_rolling_v2_pls-deep-lwr.png",bbox_inches='tight')

In [ ]:
# std-deep-lwr subset

p = re.compile('(std-deep\d-knn)')
subset = take_subset_by_re(scores_df_total ,p,reverse=False)
fig, ax = plt.subplots()

ax.set_xlabel("Stream Index")
ax.set_ylabel("R2")
ax.set_title("Streaming performance ")


for (columnName, columnData) in subset.iteritems():
    ax.plot(columnData.index,columnData,label =  f"{columnName}")
#ax.plot([len(pretest_data),len(pretest_data)],[-1,1],c="black",ls='--')

ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
plt.savefig(log_dir / f"r2_plot_std-deep-lwr.png",bbox_inches='tight')
ax.set_ylim(0,1)
plt.savefig(log_dir / f"r2_plot_v2_std-deep-lwr.png",bbox_inches='tight')


subset = take_subset_by_re(scores_df_rolling ,p,reverse=False)
fig, ax = plt.subplots()

ax.set_xlabel("Stream Index")
ax.set_ylabel("R2")
ax.set_title("Streaming performance (rolling average) ")

for (columnName, columnData) in subset.iteritems():
    ax.plot(columnData.index,columnData,label =  f"{columnName}")
#ax.plot([len(pretest_data),len(pretest_data)],[-1,1],c="black",ls='--')

ax.legend(loc='upper right',bbox_to_anchor=(1.4, 1))
plt.savefig(log_dir / f"r2_plot_rolling_std-deep-lwr.png",bbox_inches='tight')
ax.set_ylim(0,1)
plt.savefig(log_dir / f"r2_plot_rolling_v2_std-deep-lwr.png",bbox_inches='tight')


In [ ]:
id1 = 87217-len(pretest_data)
x1,y1=stream_data[id1]
river_models[f'std-deep0'].predict_one(numpy2dict(x1))